## Run comparisons by analysis type
This notebook illustrates the trajectories of individual runs and compares these for each of the analysis approaches.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import pandas as pd
from emutools.tex import DummyTexDoc, StandardTexDoc, add_image_to_doc
from aust_covid.calibration import get_targets
from aust_covid.plotting import plot_multi_spaghetti
from inputs.constants import RUN_IDS, RUNS_PATH, SUPPLEMENT_PATH
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'multi_spaghetti', 'Sequential run plots', 'austcovid')
spaghettis = {k: pd.read_hdf(RUNS_PATH / v / 'output/results.hdf', 'spaghetti') for k, v in RUN_IDS.items()}
targets = get_targets(DummyTexDoc())

In [ ]:
output = 'notifications_ma'
cases_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
cases_fig

In [ ]:
caption = 'Sequential model runs compared against targets as individually coloured lines, one panel for each approach to analysis.'
add_image_to_doc(cases_fig.update_layout(showlegend=False), 'case_spaghetti', 'svg', 'Case time series comparison by run.', app_doc, 'Calibration results', caption=caption)

In [ ]:
output = 'deaths_ma'
deaths_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
deaths_fig

In [ ]:
add_image_to_doc(deaths_fig.update_layout(showlegend=False), 'death_spaghetti', 'svg', 'Deaths time series comparison by run.', app_doc, 'Calibration results', caption=caption)

In [ ]:
output = 'adult_seropos_prop'
seropos_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
seropos_fig

In [ ]:
add_image_to_doc(seropos_fig.update_layout(showlegend=False), 'seropos_spaghetti', 'svg', 'Seropositive proportion comparison by run.', app_doc, 'Calibration results', caption=caption)

In [ ]:
description = "Figures \\ref{case_spaghetti}, \\ref{death_spaghetti} and \\ref{seropos_spaghetti} display randomly selected runs from each candidate analysis's calibration chain."
app_doc.add_line(description, 'Calibration results')

In [ ]:
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y update
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape multi_spaghetti
    ! pdflatex -shell-escape multi_spaghetti
    ! pdflatex -shell-escape multi_spaghetti